In [2]:
# Import packages 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import statsmodels.stats.multitest as smm
from pathlib import Path
from matplotlib.ticker import FixedFormatter, FixedLocator

# 1 Import Data

In [3]:
# Import DEG data 
# Define the file paths
file_paths = {
    'na_up': '../9_1_dea_lncrna/9_1_2_deg_geneid/deg_data/deg_normoxia_to_anoxia_up.csv',
    'na_down': '../9_1_dea_lncrna/9_1_2_deg_geneid/deg_data/deg_normoxia_to_anoxia_down.csv',
    'nr_up': '../9_1_dea_lncrna/9_1_2_deg_geneid/deg_data/deg_normoxia_to_reoxygenation_up.csv',
    'nr_down': '../9_1_dea_lncrna/9_1_2_deg_geneid/deg_data/deg_normoxia_to_reoxygenation_down.csv',
    'ar_up': '../9_1_dea_lncrna/9_1_2_deg_geneid/deg_data/deg_anoxia_to_reoxygenation_up.csv',
    'ar_down': '../9_1_dea_lncrna/9_1_2_deg_geneid/deg_data/deg_anoxia_to_reoxygenation_down.csv'
}

# Read the data files
dfs = {key: pd.read_csv(path) for key, path in file_paths.items()}

# Concatenate DataFrames into a new dictionary
deg = {
    'deg_na': pd.concat([dfs['na_up'], dfs['na_down']]),
    'deg_nr': pd.concat([dfs['nr_up'], dfs['nr_down']]),
    'deg_ar': pd.concat([dfs['ar_up'], dfs['ar_down']])
}

# Import GO data 
# Geneid to GO term
goterm = pd.read_csv('ccar2swissprotGO_long.txt', sep='\t') 
# rename columns
goterm.columns = ['gene_id', 'id']
# GO term to description
godesc = pd.read_csv('go_data.txt', sep='\t')
godesc.set_index('id', inplace=True)


In [4]:
pd.concat([dfs['na_up'], dfs['na_down']])

,gene_id,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,regulation,condition,-log10p,color
0,strg.7,8.583446,1.668757,0.377595,3.417007,6.331374e-04,2.788787e-03,up,Normoxia to Anoxia,2.554585,Differentially Expressed
1,strg.16,118.239224,1.568549,0.245600,4.845425,1.263410e-06,7.623835e-06,up,Normoxia to Anoxia,5.117827,Differentially Expressed
2,strg.18,144.498891,0.945254,0.197304,2.872428,4.073305e-03,1.588934e-02,up,Normoxia to Anoxia,1.798894,Differentially Expressed
3,strg.22,15.908223,1.697089,0.358630,3.676703,2.362677e-04,1.100085e-03,up,Normoxia to Anoxia,2.958574,Differentially Expressed LncRNA
4,strg.36,182.533076,1.643123,0.122581,10.316574,5.930179e-25,1.143458e-23,up,Normoxia to Anoxia,22.941780,Differentially Expressed
...,...,...,...,...,...,...,...,...,...,...,...
5919,ccar_ub25-g45078,40.030600,-0.978585,0.216295,-2.774327,5.531604e-03,2.103304e-02,up,Normoxia to Anoxia,1.677098,Differentially Expressed
5920,ccar_ub25-g45127,133.654316,-1.378683,0.102964,-9.713824,2.632712e-22,4.446298e-21,up,Normoxia to Anoxia,20.352001,Differentially Expressed
5921,ccar_ub25-g45350,7.695890,-1.573564,0.436254,-2.739352,6.156047e-03,2.320914e-02,up,Normoxia to Anoxia,1.634341,Differentially Expressed
5922,ccar_ub25-g45355,46.443179,-1.318460,0.209459,-4.487507,7.206132e-06,4.014789e-05,up,Normoxia to Anoxia,4.396337,Differentially Expressed


# Match GO term to gene_id before GO enrichment analysis

# 3 GO analysis output

In [5]:
partner = pd.read_csv('../10_lncRNA_classification/GO_analysis_input/lncRNA_deg_geneid.csv')


In [6]:

interaction = pd.read_csv('11_functional_annotation/lncRNA_deg_geneid_GOterms_test.csv')

# total number of genes in the background dataset 
total_num_genes = pd.read_csv('../A_annotation/carcar_annotation_v5_gene_lengths.csv', sep=',')

In [7]:
interaction

,category,over_represented_pvalue,under_represented_pvalue,numDEInCat,numInCat,term,ontology
1,GO:0099645,0.000061,0.999992,8,68,NaN,NaN
2,GO:0007156,0.000295,0.999892,20,393,NaN,NaN
3,GO:0051930,0.000401,0.999964,5,32,NaN,NaN
4,GO:0003964,0.000413,0.999870,15,273,NaN,NaN
5,GO:0016339,0.000535,0.999888,9,111,NaN,NaN
...,...,...,...,...,...,...,...
19121,GO:0090132,1.000000,0.980289,0,1,NaN,NaN
19122,GO:0070861,1.000000,0.979817,0,1,NaN,NaN
19123,GO:0071955,1.000000,0.979817,0,1,NaN,NaN
19124,GO:0097069,1.000000,0.935484,0,3,NaN,NaN


In [8]:
interaction['category'] = interaction['category'].str.strip()

In [9]:
interaction['term'] = godesc.loc[interaction['category']]['name'].to_numpy()

In [10]:
interaction['ontology'] = godesc.loc[interaction['category']]['namespace'].to_numpy()

In [11]:
interaction['expectedDEInCat'] = (interaction['numInCat']/len(total_num_genes))*len(partner)

In [12]:
interaction['foldEnrichment'] = interaction['numDEInCat']/interaction['expectedDEInCat']

In [13]:
interaction

,category,over_represented_pvalue,under_represented_pvalue,numDEInCat,numInCat,term,ontology,expectedDEInCat,foldEnrichment
1,GO:0099645,0.000061,0.999992,8,68,neurotransmitter receptor localization to post...,BP,1.596251,5.011743
2,GO:0007156,0.000295,0.999892,20,393,homophilic cell adhesion via plasma membrane a...,BP,9.225393,2.167929
3,GO:0051930,0.000401,0.999964,5,32,regulation of sensory perception of pain,BP,0.751177,6.656221
4,GO:0003964,0.000413,0.999870,15,273,RNA-directed DNA polymerase activity,MF,6.408479,2.340649
5,GO:0016339,0.000535,0.999888,9,111,calcium-dependent cell-cell adhesion via plasm...,BP,2.605645,3.454039
...,...,...,...,...,...,...,...,...,...
19121,GO:0090132,1.000000,0.980289,0,1,epithelium migration,BP,0.023474,0.000000
19122,GO:0070861,1.000000,0.979817,0,1,regulation of protein exit from endoplasmic re...,BP,0.023474,0.000000
19123,GO:0071955,1.000000,0.979817,0,1,recycling endosome to Golgi transport,BP,0.023474,0.000000
19124,GO:0097069,1.000000,0.935484,0,3,cellular response to thyroxine stimulus,BP,0.070423,0.000000


In [14]:
interaction['padj'] = smm.multipletests(interaction['over_represented_pvalue'],method='bonferroni')[1]

In [15]:
interaction['-log10(padj)'] = -np.log10(interaction['padj'])

In [16]:
interaction = interaction.sort_values(by='padj')

In [17]:
interaction.head(15)

,category,over_represented_pvalue,under_represented_pvalue,numDEInCat,numInCat,term,ontology,expectedDEInCat,foldEnrichment,padj,-log10(padj)
1,GO:0099645,0.000061,0.999992,8,68,neurotransmitter receptor localization to post...,BP,1.596251,5.011743,1.0,-0.0
12753,GO:0090175,1.000000,0.835983,0,9,regulation of establishment of planar polarity,BP,0.211269,0.000000,1.0,-0.0
12752,GO:0090170,1.000000,0.852789,0,8,regulation of Golgi inheritance,BP,0.187794,0.000000,1.0,-0.0
12751,GO:0090168,1.000000,0.795717,0,11,Golgi reassembly,BP,0.258217,0.000000,1.0,-0.0
12750,GO:0090158,1.000000,0.850245,0,8,endoplasmic reticulum membrane organization,BP,0.187794,0.000000,1.0,-0.0
12749,GO:0090156,1.000000,0.751427,0,14,intracellular sphingolipid homeostasis,BP,0.328640,0.000000,1.0,-0.0
12748,GO:0090150,1.000000,0.830220,0,9,establishment of protein localization to membrane,BP,0.211269,0.000000,1.0,-0.0
12754,GO:0090176,1.000000,0.835626,0,9,microtubule cytoskeleton organization involved...,BP,0.211269,0.000000,1.0,-0.0
12747,GO:0090141,1.000000,0.450071,0,40,positive regulation of mitochondrial fission,BP,0.938971,0.000000,1.0,-0.0
12745,GO:0090131,1.000000,0.887428,0,6,mesenchyme migration,BP,0.140846,0.000000,1.0,-0.0


In [18]:
interaction = interaction[interaction['padj'] <= 0.05]

In [19]:
interaction

,category,over_represented_pvalue,under_represented_pvalue,numDEInCat,numInCat,term,ontology,expectedDEInCat,foldEnrichment,padj,-log10(padj)
